In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import warnings
from tqdm.notebook import tqdm

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img, array_to_img
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

warnings.filterwarnings('ignore')

## Load the files

In [ ]:
BASEDIR = "/kaggle/input/anime-faces/data"

In [ ]:
# load complete image path to list
image_paths = []
for imagename in os.listdir(BASEDIR):
    image_path = os.path.join(BASEDIR, imagename)
    image_paths.append(image_path)

In [ ]:
print(image_path)

In [ ]:
image_paths[:5]

In [ ]:
len(image_paths)

In [ ]:
# remove unnecessary file
image_paths.remove('/kaggle/input/anime-faces/data/data')

In [ ]:
len(image_paths)

# Visualize the Image Dataset

In [ ]:
# to display grid of images(7x7)
plt.figure(figsize = (20,20))
tempimages = image_paths[:49]
index = 1

for image_path in tempimages:
    plt.subplot(7,7,index)
    #load the image
    img = load_img(image_path)
    #convert to numpy array
    img = np.array(img)
    plt.imshow(img)
    plt.axis('off')
    #increment the index for next image
    index += 1

In [ ]:
# Load the image and convert to numpy array
trainimages = [np.array(load_img(path)) for path in tqdm(image_paths)]   # pehla vala np.array ek image ke pixel ka numpy array bana ta hai
print(trainimages[0])
trainimages = np.array(trainimages) #  dusra vala  saari images  ka numpy array banata hai

In [ ]:
print(trainimages[0])

In [ ]:
trainimages.shape

In [ ]:
trainimages[0].shape

In [ ]:
#reshape the array 
trainimages = trainimages.reshape(trainimages.shape[0],64,64,3).astype('float32')

In [ ]:
# normalize the images
trainimages = (trainimages - 127.5) / 127.5

# yahan par hum normalization mei values ko (-1 to 1) kar rahe hain (0 to 1) nahi kar rahe hain kyunki hum activation function function "tanh" use karenge jo ki -1 to 1 demand karta hai

In [ ]:
trainimages[0]

# Create Generator and Discriminator

In [ ]:
# latent dimension for random noise

LatentDIM = 100   # agar latentdim kam rakhegne jaise ki 10 toh image ke produce hone ki variety less hogi and aga jayada rakhenge jaise 200 toh variety increase hogi

# weight initializer
WeightInit = keras.initializers.RandomNormal(mean = 0.0 , stddev = 0.02)

# number of channels of the image
Channels = 3  # because of RGB

# Mean = 0.0 → "Average salary" zero rakho
# Stddev = 0.02 → "Salary range" bahut chhota rakho (±0.04 ke around)

# Galat Tarika : Kisi ko ₹1,00,000 salary, kisi ko ₹500 dena → System fail!
# Sahi Tarika (stddev=0.02) : Sabko ₹9,800-₹10,200 ke beech salary dena → Stable system
# Zyada bade weights → Network fail ho jata hai, Chhote weights se model aram se seekhta hai

# Kyun Mean=0? = Balance Ke Liye: Positive/Negative dono tarah ke connections ban paaye ,Jaise company me HR aur IT dono departments ka balance

# Generator Model

Generator model will create new images similar to training data from random noise

In [ ]:
model = Sequential(name = 'generator')

# 1d random noise

model.add(layers.Dense(8 * 8 * 512, input_dim = LatentDIM))

#model.add(layers.BatchNormalization())
model.add(layers.ReLU())

# convert 1d to 3d
model.add(layers.Reshape((8,8,512)))


# umsample to 16 x 16

model.add(layers.Conv2DTranspose(256, (4,4), strides = (2,2), padding = 'same',kernel_initializer = WeightInit))          
#model.add(layers.BatchNormalization())
model.add(layers.ReLU())


# umsample to 32 x 32

model.add(layers.Conv2DTranspose(128, (4,4), strides = (2,2), padding = 'same',kernel_initializer = WeightInit))          
#model.add(layers.BatchNormalization())
model.add(layers.ReLU())

# umsample to 64 x 64

model.add(layers.Conv2DTranspose(64, (4,4), strides = (2,2), padding = 'same',kernel_initializer = WeightInit))          
#model.add(layers.BatchNormalization())
model.add(layers.ReLU())


model.add(layers.Conv2D(Channels, (4,4), padding ='same', activation = 'tanh'))

generator = model
generator.summary()

# conv2DTranspose() upsampling ke liye hota hai jabki conv2d downsampling ke liye use hoti hai
# Yeh 2 lines ka matlab hai:

# Pehle ek dense layer se 1D noise vector (size LatentDIM, jaise 100) ko convert kiya 8×8×512 neurons mein (matlab total 32768 units).

# Phir usko 3D tensor mein reshape kiya: (8, 8, 512) → jaise image feature map ban gaya ho.

### Discriminator Model

Discriminator model will classify the images from the generator to check whether it is real or fake images

In [ ]:
model = Sequential(name = 'discriminator')
input_shape = (64,64,3)

alpha = 0.2


# create conv layers
model.add(layers.Conv2D(64, (4,4), strides = (2,2), padding = 'same', input_shape = input_shape))
model.add(layers.BatchNormalization())
model.add(layers.LeakyReLU(alpha = alpha))

# downsampling the images
model.add(layers.Conv2D(128, (4,4), strides = (2,2), padding = 'same', input_shape = input_shape))
model.add(layers.BatchNormalization())
model.add(layers.LeakyReLU(alpha = alpha))

model.add(layers.Conv2D(128, (4,4), strides = (2,2), padding = 'same', input_shape = input_shape))
model.add(layers.BatchNormalization())
model.add(layers.LeakyReLU(alpha = alpha))

model.add(layers.Flatten())
model.add(layers.Dropout(0.3))


# output class 
model.add(layers.Dense(1,activation = 'sigmoid'))

discriminator = model
discriminator.summary()

## Create DCGAN

In [ ]:
class DCGAN(keras.Model):
    def __init__(self, generator, discriminator, LatentDIM):
        super().__init__()
        self.generator = generator
        self.discriminator = discriminator
        self.LatentDIM = LatentDIM
        self.g_loss_metric = keras.metrics.Mean(name = 'g_loss')
        self.d_loss_metric = keras.metrics.Mean(name = 'd_loss')

    @property
    def metrics(self):
        return [self.g_loss_metric, self.d_loss_metric]

    def compile(self, g_optimizer, d_optimizer, loss_fn):
        super(DCGAN, self).compile()
        self.g_optimizer = g_optimizer
        self.d_optimizer = d_optimizer
        self.loss_fn = loss_fn

    def train_step(self, real_images):
        # get batch size from data
        batch_size = tf.shape(real_images)[0]

        #generate random noise
        random_noise = tf.random.normal(shape = (batch_size, self.LatentDIM))

        # train the discriminator with real(1) and fake(0) images
        with tf.GradientTape() as tape:
            #compute loss on real images
            pred_real = self.discriminator(real_images, training = True)

            # generate real images labels
            real_labels = tf.ones((batch_size, 1))
            # label smoothing
            real_labels += 0.05 * tf.random.uniform(tf.shape(real_labels))

            d_loss_real = self.loss_fn(real_labels, pred_real)

            #compute loss on fake images
            fake_images = self.generator(random_noise)
            pred_fake = self.discriminator(fake_images, training = True)

            #generate fake labels
            fake_labels = tf.zeros((batch_size,1))
            d_loss_fake = self.loss_fn(fake_labels,pred_fake)

            #Agar fake pe bhi noise daaloge (0 ± 0.05), toh generator confuse hoga ki "thoda fake chal jayega" . real_labels pe chal jaata hai kyunuki thodi kum real image chalegi.

            # total discriminator loss 
            d_loss = (d_loss_real + d_loss_fake) / 2


        # compute discriminator gradients
        gradients = tape.gradient(d_loss, self.discriminator.trainable_variables)

        # update the gradients
        self.d_optimizer.apply_gradients(zip(gradients, self.discriminator.trainable_variables))

        # train generator model
        labels = tf.ones((batch_size,1))

        # generator wants discriminator to think that fake images are real
        with tf.GradientTape() as tape:
            #generate fake images from generator
            fake_images = self.generator(random_noise,training = True)

            # classify images as real or fake
            pred_fake = self.discriminator(fake_images,training = True)

            # compute loss
            g_loss = self.loss_fn(labels, pred_fake)

        # compute gradients
        gradients = tape.gradient(g_loss, self.generator.trainable_variables)

        # update gradients
        self.g_optimizer.apply_gradients(zip(gradients, self.generator.trainable_variables))

        # update states for both models
        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)


        return {'d_loss' : self.d_loss_metric.result(),'gloss': self.g_loss_metric.result()}
            
            
        

In [ ]:
class DCGANMonitor(keras.callbacks.Callback):
    def __init__(self, num_imgs= 25, latent_dim = 100):
        self.num_imgs = num_imgs
        self.latent_dim = latent_dim

        #create random noise for generating images
        self.noise = tf.random.normal((25, latent_dim))

    def on_epoch_end(self, epoch,logs = None):
        # generate images from noise
        g_img = self.model.generator(self.noise)
        # denormalise images 
        g_img = (g_img * 127.5) + 127.5
        g_img.numpy()

        fig = plt.figure(figsize=(8,8))
        for i in range(self.num_imgs):
            plt.subplot(5,5,i+1)
            img = array_to_img(g_img[i])
            plt.imshow(img)
            plt.axis('off')

        plt.show()

    def on_train_end(self,logs = None):
        self.model.generator.save('generator.h5')

In [ ]:
dcgan = DCGAN( generator = generator ,discriminator = discriminator, LatentDIM=LatentDIM)

In [ ]:
D_LR = 0.0001
G_LR = 0.0003

#generator ko yahan faster train kara ja raha hai jayada learning rate de kar kyunki discrimintor ka learning rate jyadda hogaya toh voh generator ko hamesha supress kardiya karega and generator kabhi discriminator ko fool nhi kar payega

dcgan.compile(g_optimizer = Adam(learning_rate = G_LR, beta_1 = 0.5), d_optimizer = Adam(learning_rate = D_LR, beta_1 =0.5),loss_fn = BinaryCrossentropy())

In [ ]:
N_EPOCHS = 40
dcgan.fit(trainimages, epochs = N_EPOCHS, callbacks = [DCGANMonitor()])

## Generate New Anime Image

In [ ]:

noise = tf.random.normal((25, 100))
fig = plt.figure(figsize = (2,2))
 # generate images from noise
g_img = dcgan.generator(noise)
# denormalise images 
g_img = (g_img * 127.5) + 127.5
g_img.numpy()
    
           

img = array_to_img(g_img[0])
plt.imshow(img)
plt.axis('off')

plt.show()

In [ ]:
noise = tf.random.normal((25, 100))
fig = plt.figure(figsize = (3,3))
 # generate images from noise
g_img = dcgan.generator(noise)
# denormalise images 
g_img = (g_img * 127.5) + 127.5
g_img.numpy()
    
           

img = array_to_img(g_img[0])
plt.imshow(img)
plt.axis('off')

plt.show()

In [ ]:
model.save("dcgan_model.h5")


In [ ]:
import tensorflow as tf

# Load the saved Keras model
model = tf.keras.models.load_model("dcgan_model.h5")

# Convert to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the converted model
with open("dcgan_model.tflite", "wb") as f:
    f.write(tflite_model)


In [1]:
import tensorflow as tf
print("tensorflow version of model = ",tf.__version__)


tensorflow version of model =  2.17.1


In [2]:
import keras
print("keras version of model = ",keras.__version__)


keras version of model =  3.5.0
